In [ ]:
from IPython import get_ipython
from matplotlib import pyplot as plt
import numpy as np

from caiman.base.rois import register_multisession
from caiman.utils import visualization
from caiman.utils.utils import download_demo

from labrotation import file_handling as fh

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

import scipy

## Open (matlab) files

In [ ]:
files_list = []
while True:
    fpath = fh.open_file("Open matlab file, or press Cancel to finish")
    if fpath == ".":  # user pressed cancel
        break
    else:
        files_list.append(fpath)

In [ ]:
Y_list = []
A_list = []
dims_list = []  # Cn entry in workspace # TODO: A_sparse always have lower resolution, probably from cropping... should I define that as dims?
templates = []
for fpath in files_list:
    mat = scipy.io.loadmat(fpath)
    Y = mat["caim"][0][0][0]  # should be Y, at least... Check in matlab (opening the .mat as workspace, check struct entries)
    A_sparse = mat["caim"][0][0][1]
    FOV = mat["caim"][0][0][14]
    dims = mat["caim"][0][0][6].shape

    dims_list.append(dims)
    Y_list.append(Y)
    A_list.append(A_sparse)
    templates.append(FOV)

In [ ]:
# TODO: Make sure that all recordings same dims! 
for dims in dims_list:
    print(dims)

In [ ]:
templates_cropped = []
for template in templates:
    FOV_shape = template.shape
    cropped_shape = dims_list[0]
    
    x_crop_onesided = (FOV_shape[0] - cropped_shape[0])//2
    assert 2*x_crop_onesided == FOV_shape[0] - cropped_shape[0]

    y_crop_onesided = (FOV_shape[1] - cropped_shape[1])//2
    assert 2*y_crop_onesided == FOV_shape[1] - cropped_shape[1]
    template_cropped = template[y_crop_onesided:-y_crop_onesided,x_crop_onesided:-x_crop_onesided]  # TODO: x and y swapped?
    templates_cropped.append(template_cropped)

## Use `register_multisession()`

The function `register_multisession()` requires 3 arguments:
- `A`: A list of ndarrays or scipy.sparse.csc matrices with (# pixels X # component ROIs) for each session
- `dims`: Dimensions of the FOV, needed to restore spatial components to a 2D image
- `templates`: List of ndarray matrices of size `dims`, template image of each session

In [ ]:
spatial_union, assignments, matchings = register_multisession(A=A_list, dims=dims, templates=templates_cropped)

The function returns 3 variables for further analysis:
- `spatial_union`: csc_matrix (# pixels X # total distinct components), the union of all ROIs across all sessions aligned to the FOV of the last session.
- `assignments`: ndarray (# total distinct components X # sessions). `assignments[i,j]=k` means that component `k` from session `j` has been identified as component `i` from the union of all components, otherwise it takes a `NaN` value. Note that for each `i` there is at least one session index `j` where `assignments[i,j]!=NaN`.
- `matchings`: list of (# sessions) lists. Saves `spatial_union` indices of individual components in each session. `matchings[j][k] = i` means that component `k` from session `j` is represented by component `i` in the union of all components `spatial_union`. In other words `assignments[matchings[j][k], j] = j`.

## Post-alignment screening

The three outputs can be used to filter components in various ways. For example we can find the components that were active in at least a given a number of sessions. For more examples, check [this script](https://github.com/flatironinstitute/CaImAn/blob/master/use_cases/eLife_scripts/figure_9/Figure_9_alignment.py) that reproduces the results of [Figure 9, as presented in our eLife paper](https://elifesciences.org/articles/38173#fig9).

In [ ]:
len(A_list)  # number of sessions

In [ ]:
assignments

In [ ]:
for i in range(len(assignments_filtered)):
    print(assignments_filtered[i])

In [ ]:
assignments[0]  # TODO check if there are first nan elements... There should be (i.e. elements that dont appear in the first neuron?)

In [ ]:
for assignment in assignments:
    if np.isnan(assignment[0]):
        print(assignment)  # this looks terrible...

In [ ]:
assignments_filtered.shape

In [ ]:
# TODO: below, all first indices that don't appear in the first recording are converted to 0... 
# Instead, need to take for each neuron a true or false depending on whether they fulfill the criterion... 
# And then find the proper spatial component (from one of the recordings where it actually appears) 

In [ ]:
# Filter components by number of sessions the component could be found

n_reg = 2  # minimal number of sessions that each component has to be registered in

# Use number of non-NaNs in each row to filter out components that were not registered in enough sessions
assignments_filtered = np.array(np.nan_to_num(assignments[np.sum(~np.isnan(assignments), axis=1) >= n_reg]), dtype=int);

# Use filtered indices to select the corresponding spatial components
spatial_filtered = A_list[0][:, assignments_filtered[:, 0]]

# Plot spatial components of the selected components on the template of the last session
visualization.plot_contours(spatial_filtered, templates_cropped[0]);

## Combining data of components over multiple sessions (optional)

Now that all sessions are aligned and we have a list of re-registered neurons, we can use `assignments` and `matchings` to collect traces from neurons over different sessions.

As an exercise, we can collect the traces of all neurons that were registered in all sessions. We already gathered the indices of these neurons in the previous cell in `assignments_filtered`. Assuming that traces of each session are saved in their own `CNMF` object collected in a list, we can iterate through `assignments_filtered` and use these indices to find the re-registered neurons in every session.

Note: This notebook does not include the traces of the extracted neurons, only their spatial components. As such the loop below will produce an error if you uncomment it. However, it demonstrates how to use the results of the registration to in your own analysis to extract the traces of the same neurons across different sessions.

In [ ]:
# traces = np.zeros(assignments_filtered.shape, dtype=np.ndarray)
# for i in range(traces.shape[0]):
#     for j in range(traces.shape[1]):
#         traces[i,j] = cnm_list[j].estimates.C[int(assignments_filtered[i,j])]

Now we have the array `traces`, where element `traces[i,j] = k` is the temporal component of neuron `i` at session `j`. This can be performed with `F_dff` data or `S` spikes as well.